# [NTDS'18] milestone 2: network models
[ntds'18]: https://github.com/mdeff/ntds_2018

[Hermina Petric Maretic](https://people.epfl.ch/hermina.petricmaretic), [EPFL LTS4](https://lts4.epfl.ch)

## Students

* Team: `28`
* Students: `Guillain, Léonore Valentine; Pase, Francesco; Rusu, Cosmin-Ionut; Zhuang, Ying`
* Dataset: `Flight Routes`

## Rules

* Milestones have to be completed by teams. No collaboration between teams is allowed.
* Textual answers shall be short. Typically one to two sentences.
* Code has to be clean.
* In the first part, you cannot import any other library than we imported. In the second part, you are allowed to import any library you want.
* When submitting, the notebook is executed and the results are stored. I.e., if you open the notebook again it should show numerical results and plots. We won't be able to execute your notebooks.
* The notebook is re-executed from a blank state before submission. That is to be sure it is reproducible. You can click "Kernel" then "Restart & Run All" in Jupyter.

## Objective

The purpose of this milestone is to explore various random network models, analyse their properties and compare them to your network. In the first part of the milestone you will implement two random graph models and try to fit them to your network. In this part you are not allowed to use any additional package. In the second part of the milestone you will choose a third random graph model that you think shares some properties with your network. You will be allowed to use additional packages to construct this network, but you must explain your network choice. Finally, make your code as clean as possible, and keep your textual answers short.

## Part 0

Import the adjacency matrix of your graph that you constructed in milestone 1, as well as the number of nodes and edges of your network.

In [ ]:
import numpy as np
import pandas as pd

adjacency = np.load('adjacency.npy') # the adjacency matrix
n_nodes =  adjacency.shape[0] # the number of nodes in the network
n_edges =  np.where(adjacency > 0, 1, 0).sum() / 2 # the number of edges in the network

## Part 1

**For the computation of this part of the milestone you are only allowed to use the packages that have been imported in the cell below.**

In [ ]:
%matplotlib inline

import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy

### Question 1

Create a function that constructs an Erdős–Rényi graph.

In [ ]:
def erdos_renyi(n, p, seed = None):
    """Create an instance from the Erdos-Renyi graph model.
    
    Parameters
    ----------
    n: int
        Size of the graph.
    p: float
        Edge probability. A number between 0 and 1.
    seed: int (optional)
        Seed for the random number generator. To get reproducible results.
    
    Returns
    -------
    adjacency
        The adjacency matrix of a graph.
    """
    
    # Your code here.
 
    
    
    np.random.seed(seed)
    adjacency = np.zeros((n,n))
    
    for i in range(n):
        # add the +1 to avoid selfloops
        adjacency[i, i+1:] = np.random.binomial(1, p, n-i-1)
        adjacency[i+1:, i] = adjacency[i, i+1:]
    
    
    return adjacency

In [ ]:
er = erdos_renyi(5, 0.6, 9765)
plt.spy(er)
plt.title('Erdos-Renyi (5, 0.6)')

In [ ]:
er = erdos_renyi(10, 0.4, 7648)
plt.spy(er)
plt.title('Erdos-Renyi (10, 0.4)')

### Question 2

Use the function to create a random Erdos-Renyi graph. Choose the parameters such that number of nodes is the same as in your graph, and the number of edges similar. You don't need to set the random seed. Comment on your choice of parameters.

In [ ]:
p = 2*n_edges/(n_nodes*(n_nodes-1))
p

In [ ]:
erdos_renyi(n_nodes, p).sum()/2

In [ ]:
er_graph = erdos_renyi(n_nodes, p)
plt.spy(er_graph)

**Your answer here.**

We calcuate p so that the random er_graph we create have the similar number of edges as in our network.

$L=p* N(N-1)/2$

The result we have corresponds to our network.

### Question 3

Create a function that constructs a Barabási-Albert graph.

In [ ]:
def barabasi_albert(n, m, seed=None, m_0 = None):
    """Create an instance from the Barabasi-Albert graph model.
    
    Parameters
    ----------
    n: int
        Size of the graph.
    m: int
        Number of edges to attach from a new node to existing nodes.
    seed: int (optional)
        Seed for the random number generator. To get reproducible results.
    m_0: int (optinal)
        The number of nodes in the initial connected component
    
    Returns
    -------
    adjacency
        The adjacency matrix of a graph.
    """
    
    np.random.seed(seed)
    
    #to ensure scale free network property
    m_0 = max(m, 2)

    # Initial connected network
    adjacency = np.zeros((n,n))
    

    
    adjacency[:m_0, :m_0] = erdos_renyi(m_0, 1./m_0 , seed)
    #Should it also be connected ??
    for i in range(m_0-1):
        adjacency[i, i+1] = 1;
        adjacency[i+1, i] = 1;
    
    
    #make sure we have at least one link per node
   
    '''
    for i in range(m_0):
        if np.where(adjacency[i, :m_0-i] == 0, 0, 1).sum() == 0:
                position = np.random.randint(0, m_0)
                adjacency[i, position] = 1
                adjacency[position, i] = 1
    '''
    
    #iterate:
    for i in range(m_0, n):
        degrees = adjacency[:i].sum(axis = 1)
        total = degrees.sum()
        
        new_links = np.random.choice(i, size=m, replace=False, p = degrees / total)
        adjacency[i, new_links] = 1.
        adjacency[new_links, i] = 1.
    
    return adjacency

In [ ]:
#bad see, is all zero
ba = barabasi_albert(5, 1, 908)
plt.spy(ba)
plt.title('Barabasi-Albert (5, 1)')

In [ ]:
ba = barabasi_albert(10, 2, 8708)
plt.spy(ba)
plt.title('Barabasi-Albert (10, 2)')

### Question 4

Use the function to create a random Barabási-Albert graph. Choose the parameters such that number of nodes is the same as in your graph, and the number of edges similar. You don't need to set the random seed. Comment on your choice of parameters.

In [ ]:
avg_degree = np.where(adjacency > 0, 1, 0).sum(axis = 1).mean()
avg_degree

In [ ]:
# Your code here.
ba_graph = barabasi_albert(n_nodes, int(avg_degree))
plt.spy(ba_graph)

In [ ]:
ba_graph.sum()/2

In [ ]:
# Your code here.
#mt + m0 links -> t= n-m_0 m*(n-m)+m -> m*n-m*m+m -> m*(n+1)-m**2
# e = m*(n+1)-m**2
ba_graph = barabasi_albert(n_nodes, 6)
plt.spy(ba_graph)

In [ ]:
ba_graph.sum()/2

**Your answer here**

We calcuate m so that the random er_graph we create have the similar number of edges as in our network.

we have $mt + m0$ links, with $t= n-m_0$ 

with $t$ we obtain $e= m*(n+1)-m^2$



### Question 5

Compare the number of edges in all three networks (your real network, the Erdős–Rényi network, and the Barabási-Albert netowk).

In [ ]:
# Your code here.
print(f"Number of edges in the real network: {n_edges}")
print(f"Number of edges in the Erdős–Rényi network: {er_graph.sum() / 2}")
print(f"Number of edges in the Barabási-Albert network: {ba_graph.sum() / 2}")

### Question 6

Implement a function that computes the [Kullback–Leibler (KL) divergence](https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence) between two probability distributions.
We'll use it to compare the degree distributions of networks.

In [ ]:
def kl_divergence(p, q):
    """Compute the KL divergence between probability distributions of degrees of two networks.
    
    Parameters
    ----------
    p: np.array
        Probability distribution of degrees of the 1st graph.
    q: np.array
        Probability distribution of degrees of the 2nd graph.
    
    Returns
    -------
    kl
        The KL divergence between the two distributions.
    """
    assert p.shape == q.shape
    
    kl = 0
    for x, p_x in enumerate(p):
        kl += p_x*np.log(p_x/q[x])
    
    return kl

In [ ]:
p_test = np.array([0.2, 0.2, 0.2, 0.4])
q_test = np.array([0.3, 0.3, 0.1, 0.3])
kl_divergence(p_test, q_test)

### Question 7

Compare the degree distribution of your network to each of the two synthetic ones, in terms of KL divergence. **Hint:** Make sure you normalise your degree distributions to make them valid probability distributions.

In [ ]:
# Your code here.

#calculate degree distrubution for each graph
degrees = np.where(adjacency > 0, 1, 0).sum(axis = 1)
degree_network = np.histogram(degrees, bins=(degrees.max()+1)-degrees.min())
degree_network = degree_network[0]

degrees_er = er_graph.sum(axis = 1) 
degree_er_network = np.histogram(degrees_er, bins=int(degrees_er.max()+1-degrees_er.min()))
degree_er_network = degree_er_network[0]

degrees_ba = ba_graph.sum(axis = 1)
degree_ba_network = np.histogram(degrees_ba, bins=int(degrees_ba.max()+1-degrees_ba.min()))
degree_ba_network = degree_ba_network[0]

min_s = [degrees.min(), degrees_er.min(), degrees_ba.min()]
max_s = [degrees.max(), degrees_er.max(), degrees_ba.max()]
degree_networks = [degree_network, degree_er_network, degree_ba_network]

In [ ]:
#pad arrays to have same length

for i, s in enumerate(min_s):
    before_pad = int(s - min(min_s))
    after_pad = int( max(max_s) - max_s[i])
    degree_networks[i] = np.pad(degree_networks[i], pad_width=(before_pad, after_pad), mode='constant', constant_values = 0)

In [ ]:
#normalize

tmp = []

for n in degree_networks:
    n[n== 0] = 1
    
    #alternative smoothing
    #scp = scipy.interpolate.UnivariateSpline(range(min(min_s), max(max_s)+1), n)
    #n = scp(range(min(min_s), max(max_s)+1))
    
    N = n.shape[0]
    np.convolve(n, np.ones((N,))/N, mode='valid')
    n = n/ n.sum()
    tmp.append(n)
    
degree_networks = tmp

In [ ]:
plt.title('normalized degree distributions')
for i in degree_networks:
    plt.plot(i)
    
plt.legend(['our network', 'erdos-reyni', 'a-b'])

In [ ]:
print(f"KL Divergence between the original network and the Erdős–Rényi network: {kl_divergence(degree_networks[0], degree_networks[1])}")
print(f"KL Divergence between the original network and the Barabási-Albert network: {kl_divergence(degree_networks[0], degree_networks[2])}")

In [ ]:
tmp = []

for n in degree_networks:
    n = n[5:]
    n = n/ n.sum()
    tmp.append(n)
    
degree_networks = tmp

In [ ]:
print(f"KL Divergence between the original network and the Erdős–Rényi network: {kl_divergence(degree_networks[0], degree_networks[1])}")
print(f"KL Divergence between the original network and the Barabási-Albert network: {kl_divergence(degree_networks[0], degree_networks[2])}")

### Question 8

Plot the degree distribution historgrams for all three networks. Are they consistent with the KL divergence results? Explain.

In [ ]:
# Your code here.
plt.title('degree distribution of our network')
plt.hist(adjacency.sum(axis = 1), bins=50)
plt.xlabel('degree')
plt.ylabel('counts')
plt.show()

In [ ]:
plt.title('degree distribution of Erdős–Rényi graph')
plt.hist(er_graph.sum(axis = 1), bins=50)
plt.xlabel('degree')
plt.ylabel('counts')
plt.show()

In [ ]:
plt.title('degree distribution of Erdős–Rényi')
plt.hist(ba_graph.sum(axis = 1), bins=50)
plt.xlabel('degree')
plt.ylabel('counts')
plt.show()

In [ ]:
# Your code here.
plt.hist(er_graph.sum(axis=1), alpha=0.4, log=True)
plt.hist(ba_graph.sum(axis=1), alpha=0.4, bins=100)
plt.hist(np.where(adjacency > 0, 1, 0).sum(axis=1), alpha=0.4, bins=100)
plt.legend(['er', 'ab', 'flights'])
plt.show()

**Your answer here.**

We overlap the three degree distribution historgrams and we find that the er model has a much more similar distribution with our network than the ba model.
As the KL divergence of the er model is smaller than the ba model,so the degree distributions are consistent with the KL divergence results.


### Question 9

Imagine you got equal degree distributions. Would that guarantee you got the same graph? Explain.

# TODO(someone): Proof read my (cosmin) answer:
If we have the same degree distrubtion does not mean we have the same graph. If two graphs have same degree sequence, that does not mean that the graphs are isomorphic, hence they cannot be the same. To proof this, we will find a counter example. Consider two graphs with the same degree distribution: (2, 2, 2, 1, 1):

```
* 1 -- 2 -- 3 -- 4 -- 5
* 1 -- 2    3 -- 4
             \ /
              5
```

## Part 2

**You are allowed to use any additional library here (e.g., NetworkX, PyGSP, etc.).** Be careful not to include something here and use it in part 1!

In [ ]:
import networkx as nx

### Question 10

Choose a random network model that fits you network well. Explain your choice. 

**Hint:** Check lecture notes for different network models and their properties. Your choice should be made based on at least one property you'd expect to be similar.

**TODO(team): Check this answer**

In our network, very popular airports act as hubs. Since these airports are bigger, they are more probable to get new links (create a new direct flight). Moreover, the distribution of a generated Barabási-Albert model have the same "shape" as the one of our flight network. The chosen degree (the average degreee), however, yields a network with far more edges than we have (one reason why the KL-divergence is big). We expect that by choosing the correct paremeter, the degree distribution of the nodes will be similar.

### Question 11

Explain (in short) how the chosen model works.

**TODO(team): Check this answer**

[WikiPage](https://en.wikipedia.org/wiki/Barab%C3%A1si%E2%80%93Albert_model)


The Barabási-Albert generates a random scale-free network (the degree distrubtion follows a power law), Some examples of such networks are: Internet, World Wide Web, Citations Network or Social Networks. The graphs usually contains high degree nodes also knows as "hubs". The model incorporates two important general concepts: growth and preferential attachment. Both growth and preferential attachment exist widely in real networks. Growth means that the number of nodes in the network increase over time. Preferential attachment states that the more connected a node is, the more likely it is to receive new links. If we think in terms of our Flights Network, it is likely that if a new airport opens up in a country, they will likely create direct flights to the most popular airports so that more traffic flows through the new airport.

The algorithm is fairly simple:

1. Create a single connected component of $m0$ nodes (this will be our approximate hubs) 
2. For every new node (until we reached the number of nodes we want):
    1. Create links to ($m <= m0$) already existing nodes with probabilities proportional to the degree of the existing node, i.e.
    $$p_i = \frac{degree_i}{\sum_{j \in S}{degree_j}} \text{ where } S \text{ is the set of existing nodes} $$


### Question 12

Create a random graph from that model, such that the number of nodes is the same as in your graph.

In [ ]:
def new_barabasi_albert(n, m, seed=None, m_0 = None, alpha = 1):
    """Create an instance from the Barabasi-Albert graph model.
    
    Parameters
    ----------
    n: int
        Size of the graph.
    m: int
        Number of edges to attach from a new node to existing nodes.
    seed: int (optional)
        Seed for the random number generator. To get reproducible results.
    m_0: int (optinal)
        The number of nodes in the initial connected component
    
    Returns
    -------
    adjacency
        The adjacency matrix of a graph.
    """
    
    np.random.seed(seed)
    
    #to ensure scale free network property
    m_0 = max(m, 2)

    # Initial connected network
    adjacency = np.zeros((n,n))
    

    
    adjacency[:m_0, :m_0] = erdos_renyi(m_0, 1./m_0 , seed)
    #Should it also be connected ??
    for i in range(m_0-1):
        adjacency[i, i+1] = 1;
        adjacency[i+1, i] = 1;
    
    print (adjacency[:m_0, :m_0])
    
    
    #iterate:
    for i in range(m_0, n):
        degrees = adjacency[:i].sum(axis = 1)
        total = (degrees**alpha).sum()
        
        new_links = np.random.choice(i, size=m, replace=False, p = degrees**alpha / total)
        adjacency[i, new_links] = 1.
        adjacency[new_links, i] = 1.
    
    return adjacency

In [ ]:
# Your code here.
ba_graph = new_barabasi_albert(n_nodes, 6, None, 100, alpha = 1.05)
plt.spy(ba_graph)

In [ ]:
print(f"Our network number of nodes: {n_nodes} and edges: {n_edges}")
print(f"Barabási-Albert network number of nodes: {ba_graph.shape[0]} and edges: {ba_graph.sum() / 2}")


### Question 13

Check the properties you expected to be similar, and compare to your network.

In [ ]:
# Your code here.
one_adjacency = np.where(adjacency > 0, 1, 0)
x = one_adjacency.sum(axis = 1)
degree_network = x[x>0]
degree_ba_network = ba_graph.sum(axis = 1)


plt.hist(degree_network, bins = 237, alpha=0.4, log = True)
plt.hist(degree_ba_network, bins = 237, alpha=0.4, log = True)

plt.show()

print(f"KL Divergence between the original network and the Barabási-Albert network: {kl_divergence(degree_network, degree_ba_network)}")

In [ ]:
adjacency

In [ ]:
plt.hist(ba_graph.sum(axis=1), alpha=0.4, log = True)
plt.hist(one_adjacency.sum(axis=1), alpha=0.4, log = True)

**Your answer here.**

We see the distributions are fairly equal, altought the KL Divergence is quite big (we see there are some outliers in our random model that drives this metric up. Our original network does not have many hubs, but the Barabási-Albert model tends to create more hubs.